# Use MySQL with Pandas

In [5]:
import mysql.connector
import pandas as pd

In [8]:
con = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root",
  database="world",
  auth_plugin='mysql_native_password'
)

In [17]:
pd.read_sql("SELECT * FROM city LIMIT 10;", con=con)

,ID,Name,CountryCode,District,Population
0,1,Kabul,AFG,Kabol,1780000
1,2,Qandahar,AFG,Qandahar,237500
2,3,Herat,AFG,Herat,186800
3,4,Mazar-e-Sharif,AFG,Balkh,127800
4,5,Amsterdam,NLD,Noord-Holland,731200
5,6,Rotterdam,NLD,Zuid-Holland,593321
6,7,Haag,NLD,Zuid-Holland,440900
7,8,Utrecht,NLD,Utrecht,234323
8,9,Eindhoven,NLD,Noord-Brabant,201843
9,10,Tilburg,NLD,Noord-Brabant,193238


In [15]:
pd.read_sql("SELECT * FROM country LIMIT 10;", con=con)

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ABW,Aruba,North America,Caribbean,193.0,NaN,103000,78.4,828.0,793.0,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129,AW
1,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1,AF
2,AGO,Angola,Africa,Central Africa,1246700.0,1975.0,12878000,38.3,6648.0,7984.0,Angola,Republic,JosÃ© Eduardo dos Santos,56,AO
3,AIA,Anguilla,North America,Caribbean,96.0,NaN,8000,76.1,63.2,NaN,Anguilla,Dependent Territory of the UK,Elisabeth II,62,AI
4,ALB,Albania,Europe,Southern Europe,28748.0,1912.0,3401200,71.6,3205.0,2500.0,ShqipÃ«ria,Republic,Rexhep Mejdani,34,AL
5,AND,Andorra,Europe,Southern Europe,468.0,1278.0,78000,83.5,1630.0,NaN,Andorra,Parliamentary Coprincipality,,55,AD
6,ANT,Netherlands Antilles,North America,Caribbean,800.0,NaN,217000,74.7,1941.0,NaN,Nederlandse Antillen,Nonmetropolitan Territory of The Netherlands,Beatrix,33,AN
7,ARE,United Arab Emirates,Asia,Middle East,83600.0,1971.0,2441000,74.1,37966.0,36846.0,Al-Imarat al-Â´Arabiya al-Muttahida,Emirate Federation,Zayid bin Sultan al-Nahayan,65,AE
8,ARG,Argentina,South America,South America,2780400.0,1816.0,37032000,75.1,340238.0,323310.0,Argentina,Federal Republic,Fernando de la RÃºa,69,AR
9,ARM,Armenia,Asia,Middle East,29800.0,1991.0,3520000,66.4,1813.0,1627.0,Hajastan,Republic,Robert KotÂšarjan,126,AM


In [16]:
pd.read_sql("SELECT * FROM countrylanguage LIMIT 10;", con=con)

,CountryCode,Language,IsOfficial,Percentage
0,ABW,Dutch,T,5.3
1,ABW,English,F,9.5
2,ABW,Papiamento,F,76.7
3,ABW,Spanish,F,7.4
4,AFG,Balochi,F,0.9
5,AFG,Dari,T,32.1
6,AFG,Pashto,T,52.4
7,AFG,Turkmenian,F,1.9
8,AFG,Uzbek,F,8.8
9,AGO,Ambo,F,2.4


In [14]:
# country who has the largest percentage of population speaking english
query = """
SELECT c.name
FROM country AS c
         INNER JOIN
     (SELECT countrycode
      FROM countrylanguage
      WHERE percentage = (SELECT max(percentage) AS percentage FROM countrylanguage WHERE language = 'english')
        AND language = 'english') AS cl
     ON c.code = cl.countrycode
;"""
pd.read_sql(query, con=con)

,name
0,Bermuda


In [19]:
# percentage of chinese speaking population in the world population
query = """
SELECT concat(round((SELECT sum(population.population)
                     FROM (SELECT pbp.population * pbp.percentage / 100 AS population
                           FROM (SELECT c.population, cl.percentage
                                 FROM country AS c
                                          INNER JOIN (SELECT * FROM countrylanguage WHERE language = 'chinese') AS cl
                                                     ON c.code = cl.countrycode) AS pbp) AS population) /
                    (SELECT sum(population) FROM country) * 100, 2), '%') AS ratio;
"""
pd.read_sql(query, con=con)

,ratio
0,19.61%


In [20]:
pd.read_sql("SHOW TABLES;", con=con)

,Tables_in_world
0,city
1,country
2,countrylanguage


In [22]:
con.close()